In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, HeteroData
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
import networkx as nx

from rdkit import Chem
from rdkit.Chem import AllChem

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Input, Dense
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from spektral.layers import GATConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot, load_sdf
#from spektral.utils import nx_to_numpy
#from spektral.chem import sdf_to_nx

/home/yuyang/anaconda3/envs/SINGA/lib/python3.10/site-packages/torch/cuda/__init__.py:126: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /opt/conda/conda-bld/pytorch_1689059489744/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
2023-07-25 06:05:26.255179: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 06:05:29.346444: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import logging
from sklearn.base import BaseEstimator, TransformerMixin
logger = logging.getLogger(__name__)

class MoleculeToGraph(BaseEstimator, TransformerMixin):
    def __init__(self, file_format='sdf'):
        """Constructor.

        valid 'file_format' strings and what the transformer will expect

        smi: path to .smi file
        sdf: pat to .sdf file
        """
        self.file_format = file_format

    def transform(self, data):
        """Transform."""
        try:
            if self.file_format == 'smi':
                graphs = smi_to_nx(data)
            elif self.file_format == 'sdf':
                graphs = sdf_to_nx(data)
            else:
                raise Exception('file_format must be smi or sdf')
            for graph in graphs:
                yield graph

        except Exception as e:
            logger.debug('Failed iteration. Reason: %s' % e)
            logger.debug('Exception', exc_info=True)


################
# import to networkx graphs
###############
def load_sdf(file):
    suppl = Chem.SDMolSupplier(file)
    for mol in suppl:
        if mol:
            yield mol

def load_smi(file):
    suppl = Chem.SmilesMolSupplier(file)
    for mol in suppl:
        if mol:
            yield mol

def sdf_to_nx(file):
    # read sdf file
    suppl = load_sdf(file)
    for mol in suppl:
        yield rdkmol_to_nx(mol)

def smi_to_nx(file):
    # read smi file
    suppl = load_smi(file)
    for mol in suppl:
        yield rdkmol_to_nx(mol)

def rdkmol_to_nx(mol):
    #  rdkit-mol object to nx.graph
    graph = nx.Graph()
    for e in mol.GetAtoms():
        graph.add_node(e.GetIdx(), label=e.GetSymbol())
    for b in mol.GetBonds():
        graph.add_edge(b.GetBeginAtomIdx(), b.GetEndAtomIdx(), label=str(int(b.GetBondTypeAsDouble())))
    return graph

def smiles_strings_to_nx(smileslist):
    # smiles strings
    for smile in smileslist:
        mol = Chem.MolFromSmiles(smile)
        yield rdkmol_to_nx(mol)

################
# exporting networkx graphs
###############

def nx_to_smi(graphs, file):
    # writes smiles strings to a file
    chem = [nx_to_rdkit(graph) for graph in graphs]
    smis = [Chem.MolToSmiles(m) for m in chem]
    with open(file, 'w') as f:
        f.write("SMILES\n")
        f.write('\n'.join(smis))

def nx_to_rdkit(graph):
    m = Chem.MolFromSmiles('')
    mw = Chem.RWMol(m)
    atom_index = {}
    for n, d in graph.nodes(data=True):
        atom_index[n] = mw.AddAtom(Chem.Atom(d['label']))
    for a, b, d in graph.edges(data=True):
        start = atom_index[a]
        end = atom_index[b]
        bond_type = d.get("label", '1')
        if bond_type == '1':
            mw.AddBond(start, end, Chem.BondType.SINGLE)
        elif bond_type == '2':
            mw.AddBond(start, end, Chem.BondType.DOUBLE)
        elif bond_type == '3':
            mw.AddBond(start, end, Chem.BondType.TRIPLE)
        # more options:
        # http://www.rdkit.org/Python_Docs/rdkit.Chem.rdchem.BondType-class.html
        else:
            raise Exception('bond type not implemented')

    mol = mw.GetMol()
    return mol

In [3]:
def get_graph_info(mol):
    # Extract atom features and coordinates
    atom_features = []
    coordinates = []
    beginIdx, endIdx = [], []

    for i in range(len(mol.GetBonds())):
        beginAtomIdx = mol.GetBondWithIdx(i).GetBeginAtomIdx()
        endAtomIdx = mol.GetBondWithIdx(i).GetEndAtomIdx()
        beginIdx.append(beginAtomIdx)
        endIdx.append(endAtomIdx)
         
    bond_index = np.array([beginIdx, endIdx])
    bond_index = torch.LongTensor(bond_index)

    for atom in mol.GetAtoms():
        atom_features.append(atom.GetAtomicNum())
        coordinates.append(list(mol.GetConformer().GetAtomPosition(atom.GetIdx())))

    # Create edge index tensor (fully connected graph)
    num_atoms = len(atom_features)
    edge_index = torch.tensor([(i, j) for i in range(num_atoms) for j in range(num_atoms) if i != j], dtype=torch.long).t().contiguous()
    
    # Convert atom features and coordinates to tensors
    x = torch.tensor(atom_features, dtype=torch.float).view(-1, 1)
    pos = torch.tensor(coordinates, dtype=torch.float)

    return x, pos, edge_index, bond_index

In [4]:
def pdb_to_graph(pdb_ligand_file, pdb_protein_file):
    # Read PDB files
    mol_ligand = Chem.MolFromPDBFile(pdb_ligand_file, removeHs = False, sanitize = False)
    mol_protein = Chem.MolFromPDBFile(pdb_protein_file, removeHs = False, sanitize = False)

    # Generate 3D coordinates
    mol_ligand, mol_protein = Chem.AddHs(mol_ligand), Chem.AddHs(mol_protein)
    AllChem.EmbedMolecule(mol_ligand)
    AllChem.EmbedMolecule(mol_protein)

    x, pos, edge_index, bond_index = get_graph_info(mol_ligand)

    # Create the Data object
    #data = Data(x=x, pos=pos, edge_index=bond_index)
    data = HeteroData()
    data['ligand'].x = x
    data['ligand'].pos = pos
    #data['ligand'].edge_index = bond_index
    data['ligand', 'conc_l', 'ligand'].edge_index = bond_index

    x, pos, edge_index, bond_index = get_graph_info(mol_protein)
    data['protein'].x = x
    data['protein'].pos = pos
    #data['protein'].edge_index = bond_index
    data['protein', 'conc_p', 'protein'].edge_index = bond_index

    return data

In [5]:
# Specify the path to the PDB file
pdb_ligand_file_path = '../../download/v2020-other-PL/9icd/9icd_ligand_dock001.pdb'
pdb_protein_file_path = '../../download/v2020-other-PL/9icd/9icd_pocket.pdb'

# Convert PDB to graph data
graph_data = pdb_to_graph(pdb_ligand_file_path, pdb_protein_file_path)

# Print the graph data
print(graph_data)

[06:54:10] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[06:54:10] UFFTYPER: Warning: hybridization set to SP3 for atom 23


HeteroData(
  ligand={
    x=[54, 1],
    pos=[54, 3]
  },
  protein={
    x=[678, 1],
    pos=[678, 3]
  },
  (ligand, conc_l, ligand)={ edge_index=[2, 56] },
  (protein, conc_p, protein)={ edge_index=[2, 663] }
)


In [ ]:
# a = to_networkx(graph_data, to_undirected=True)
# nx.draw(a)

## Another approach

In [6]:
def nx_to_numpy(graph):
    # Get the adjacency matrix from the NetworkX graph
    adj_matrix = nx.adjacency_matrix(graph)

    # Convert the adjacency matrix to a dense NumPy array
    adj_array = adj_matrix.toarray()

    # Get the node labels from the NetworkX graph
    node_labels = np.array(list(nx.get_node_attributes(graph, 'label').values()))

    return adj_array, node_labels

In [7]:
sdf_loaded = Chem.SDMolSupplier("Trainingset_Delaunay-CHNO5000selected.sdf", removeHs=False, sanitize=False)
#sdf_loaded = load_sdf('Trainingset_Delaunay-CHNO5000selected.sdf')

In [ ]:
sdf_nx = rdkmol_to_nx(sdf_loaded[0])
sdf_adj, sdf_node = nx_to_numpy(sdf_nx)

uniq_node = np.unique([v for x in sdf_node for v in np.unique(x)])
node = [label_to_one_hot(x, uniq_node) for x in sdf_node]

In [ ]:
uniq_node

## Analyse protein-ligand (non-contact) interactions

In [11]:
from pathlib import Path
import time
import warnings
warnings.filterwarnings("ignore")

import nglview as nv
import openbabel
from matplotlib import colors
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from opencadd.structure.core import Structure

In [29]:
import parmed as pmd

pdb_id = '11gs'
pdb1 = pmd.load_file(f'../../download/v2020-other-PL/{pdb_id}/{pdb_id}_protein.pdb')
pdb2 = pmd.load_file(f'../../download/v2020-other-PL/{pdb_id}/{pdb_id}_ligand_dock001.pdb')

combined = pdb1 + pdb2
combined.save(f'{pdb_id}.pdb')

In [30]:
def retrieve_plip_interactions(pdb_file):
    """
    Retrieves the interactions from PLIP.

    Parameters
    ----------
    pdb_file :
        The PDB file of the complex.

    Returns
    -------
    dict :
        A dictionary of the binding sites and the interactions.
    """
    protlig = PDBComplex()
    protlig.load_pdb(pdb_file)  # load the pdb file
    for ligand in protlig.ligands:
        protlig.characterize_complex(ligand)  # find ligands and analyze interactions
    sites = {}
    # loop over binding sites
    for key, site in sorted(protlig.interaction_sets.items()):
        binding_site = BindingSiteReport(site)  # collect data about interactions
        # tuples of *_features and *_info will be converted to pandas DataFrame
        keys = (
            "hydrophobic",
            "hbond",
            "waterbridge",
            "saltbridge",
            "pistacking",
            "pication",
            "halogen",
            "metal",
        )
        # interactions is a dictionary which contains relevant information for each
        # of the possible interactions: hydrophobic, hbond, etc. in the considered
        # binding site. Each interaction contains a list with
        # 1. the features of that interaction, e.g. for hydrophobic:
        # ('RESNR', 'RESTYPE', ..., 'LIGCOO', 'PROTCOO')
        # 2. information for each of these features, e.g. for hydrophobic
        # (residue nb, residue type,..., ligand atom 3D coord., protein atom 3D coord.)
        interactions = {
            k: [getattr(binding_site, k + "_features")] + getattr(binding_site, k + "_info")
            for k in keys
        }
        sites[key] = interactions
    return sites

We create the dictionary for the complex of interest and see how many binding sites are detected:

In [68]:
interactions_by_site = retrieve_plip_interactions(f"{pdb_id}.pdb")
print(
    f"Number of binding sites detected in {pdb_id}.pdb : "
    f"{len(interactions_by_site)}\n"
    f"with {interactions_by_site.keys()}"
)

Number of binding sites detected in 11gs.pdb : 1
with dict_keys(['LIG:Z:591'])


In this case, the first binding site containing ligand 03P will be further investigated.

In [69]:
index_of_selected_site = 0
selected_site = list(interactions_by_site.keys())[index_of_selected_site]
print(selected_site)

LIG:Z:591


We can construct a ```pandas.DataFrame``` for a binding site and particular interaction type.

In [70]:
def create_df_from_binding_site(selected_site_interactions, interaction_type="hbond"):
    """
    Creates a data frame from a binding site and interaction type.

    Parameters
    ----------
    selected_site_interactions : dict
        Precaluclated interactions from PLIP for the selected site
    interaction_type : str
        The interaction type of interest (default set to hydrogen bond).

    Returns
    -------
    pd.DataFrame :
        DataFrame with information retrieved from PLIP.
    """

    # check if interaction type is valid:
    valid_types = [
        "hydrophobic",
        "hbond",
        "waterbridge",
        "saltbridge",
        "pistacking",
        "pication",
        "halogen",
        "metal",
    ]

    if interaction_type not in valid_types:
        print("!!! Wrong interaction type specified. Hbond is chosen by default!!!\n")
        interaction_type = "hbond"

    df = pd.DataFrame.from_records(
        # data is stored AFTER the column names
        selected_site_interactions[interaction_type][1:],
        # column names are always the first element
        columns=selected_site_interactions[interaction_type][0],
    )
    
    return df

In the next cell, we show the hydrophobic interactions from the selected binding site.

In [71]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="hydrophobic")

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST,LIGCARBONIDX,PROTCARBONIDX,LIGCOO,PROTCOO
0,7,PHE,A,591,LIG,Z,3.85,6740,112,"(6.689, 1.326, 30.431)","(10.072, 2.163, 32.063)"
1,9,VAL,A,591,LIG,Z,3.54,6741,149,"(8.517, 1.116, 28.027)","(11.207, -1.184, 28.013)"
2,103,ILE,A,591,LIG,Z,3.57,6734,1627,"(10.154, 3.047, 23.083)","(11.308, 4.406, 19.99)"
3,107,TYR,A,591,LIG,Z,3.84,6734,1696,"(10.154, 3.047, 23.083)","(6.599, 1.683, 22.625)"
4,107,TYR,A,591,LIG,Z,3.78,6727,1694,"(6.907, 3.074, 26.738)","(4.842, 1.213, 24.182)"


If hydrogen interactions are of interest, the table can be generated as follow:

In [72]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="hbond")

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,SIDECHAIN,DIST_H-A,DIST_D-A,DON_ANGLE,PROTISDON,DONORIDX,DONORTYPE,ACCEPTORIDX,ACCEPTORTYPE,LIGCOO,PROTCOO
0,51,LEU,A,591,LIG,Z,False,1.72,2.67,150.45,True,789,Nam,6715,O2,"(10.585, 7.113, 30.729)","(12.883, 8.479, 30.778)"
1,37,TRP,A,591,LIG,Z,True,3.64,4.04,105.47,True,591,Nar,6718,Nam,"(9.046, 8.58, 29.984)","(6.904, 8.136, 33.385)"
2,50,GLN,A,591,LIG,Z,True,3.18,3.78,117.41,True,786,Nam,6718,Nam,"(9.046, 8.58, 29.984)","(8.896, 10.638, 26.82)"
3,50,GLN,A,591,LIG,Z,False,3.13,3.48,108.29,False,6721,O2,777,O2,"(10.614, 10.885, 31.207)","(14.041, 10.31, 31.406)"
4,51,LEU,A,591,LIG,Z,False,1.64,2.62,168.35,False,6712,Nam,794,O2,"(11.05, 7.321, 27.713)","(13.511, 7.308, 28.618)"


In [73]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="waterbridge")

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST_A-W,DIST_D-W,DON_ANGLE,WATER_ANGLE,PROTISDON,DONOR_IDX,DONORTYPE,ACCEPTOR_IDX,ACCEPTORTYPE,WATER_IDX,LIGCOO,PROTCOO,WATERCOO
0,12,ARG,A,591,LIG,Z,3.68,3.12,160.48,121.20,True,199,Ng+,6703,N3,6607,"(13.421, 9.166, 22.647)","(13.664, 6.134, 22.049)","(10.628, 6.786, 22.377)"
1,12,ARG,A,591,LIG,Z,3.61,3.12,160.48,72.60,True,199,Ng+,6706,O.co2,6607,"(10.633, 10.38, 22.738)","(13.664, 6.134, 22.049)","(10.628, 6.786, 22.377)"


In [74]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="saltbridge")

,RESNR,RESTYPE,RESCHAIN,PROT_IDX_LIST,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST,PROTISPOS,LIG_GROUP,LIG_IDX_LIST,LIGCOO,PROTCOO
0,12,ARG,A,"199,202,205",591,LIG,Z,5.00,True,Carboxylate,"6706,6707","(10.472999999999999, 9.370000000000001, 22.333)","(14.368333333333332, 6.532, 20.98966666666667)"
1,43,LYS,A,689,591,LIG,Z,3.18,True,Carboxylate,"6722,6721","(9.575, 11.23, 31.326999999999998)","(6.949, 12.901, 31.966)"


In [75]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="pistacking")

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,PROT_IDX_LIST,CENTDIST,ANGLE,OFFSET,TYPE,LIG_IDX_LIST,LIGCOO,PROTCOO


In [76]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="pication")

,RESNR,RESTYPE,RESCHAIN,PROT_IDX_LIST,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,DIST,OFFSET,PROTCHARGED,LIG_GROUP,LIG_IDX_LIST,LIGCOO,PROTCOO


In [77]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="halogen")

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,SIDECHAIN,DIST,DON_ANGLE,ACC_ANGLE,DON_IDX,DONORTYPE,ACC_IDX,ACCEPTORTYPE,LIGCOO,PROTCOO


In [78]:
create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="metal")

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,METAL_IDX,METAL_TYPE,TARGET_IDX,TARGET_TYPE,COORDINATION,DIST,LOCATION,RMS,GEOMETRY,COMPLEXNUM,METALCOO,TARGETCOO


Now, let’s try to represent those interactions in the NGL viewer. We can draw cylinders between the interaction points (```LIGCOO``` and ```PROTCOO``` in the ```pandas.DataFrame```) and colour-code them as shown in ```colour_map```, which uses RGB tuples.

In [79]:
color_map = {
    "hydrophobic": [0.90, 0.10, 0.29],
    "hbond": [0.26, 0.83, 0.96],
    "waterbridge": [1.00, 0.88, 0.10],
    "saltbridge": [0.67, 1.00, 0.76],
    "pistacking": [0.75, 0.94, 0.27],
    "pication": [0.27, 0.60, 0.56],
    "halogen": [0.94, 0.20, 0.90],
    "metal": [0.90, 0.75, 1.00],
}

Define a helper function to show the interactions.

In [25]:
def show_interactions_3d(
    pdb_id, selected_site_interactions, highlight_interaction_colors=color_map
):
    """
    3D visualization of protein-ligand interactions.

    Parameters
    ----------
    pdb_id : str
        The pdb ID of interest.
    selected_site_interactions : dict
        Precaluclated interactions from PLIP for the selected site
    highlight_interaction_colors : dict
        The colors used to highlight the different interaction types.

    Returns
    -------
    NGL viewer with explicit interactions given by PLIP.
    """

    # Create NGLviewer
    viewer = nv.NGLWidget(height="600px", default=True, gui=True)
    # Add protein
    prot_component = viewer.add_pdbid(pdb_id)
    # Add the ligands
    viewer.add_representation(repr_type="ball+stick", selection="hetero and not water")

    interacting_residues = []
    for interaction_type, interaction_list in selected_site_interactions.items():
        color = highlight_interaction_colors[interaction_type]
        if len(interaction_list) == 1:
            continue
        df_interactions = pd.DataFrame.from_records(
            interaction_list[1:], columns=interaction_list[0]
        )
        for _, interaction in df_interactions.iterrows():
            name = interaction_type
            # add cylinder between ligand and protein coordinate
            viewer.shape.add_cylinder(
                interaction["LIGCOO"],
                interaction["PROTCOO"],
                color,
                [0.1],
                name,
            )
            interacting_residues.append(interaction["RESNR"])
    
    # Display interacting residues
    res_sele = " or ".join([f"({r} and not _H)" for r in interacting_residues])
    res_sele_nc = " or ".join([f"({r} and ((_O) or (_N) or (_S)))" for r in interacting_residues])
    prot_component.add_ball_and_stick(sele=res_sele, colorScheme="chainindex", aspectRatio=1.5)
    prot_component.add_ball_and_stick(sele=res_sele_nc, colorScheme="element", aspectRatio=1.5)
    
    # Center on ligand
    viewer.center("ligand")
    
    return viewer

In [28]:
viewer_3d = show_interactions_3d('combine', interactions_by_site[selected_site])
viewer_3d.render_image(trim=True, factor=2, transparent=True);
viewer_3d._display_image()

In [80]:
all_intr_types = ("hydrophobic",
                  "hbond",
                  "waterbridge",
                  "saltbridge",
                  "pistacking",
                  "pication",
                  "halogen",
                  "metal")

allPLInteractions = pd.DataFrame()
for intr_type in all_intr_types:
    _df = create_df_from_binding_site(interactions_by_site[selected_site], interaction_type=intr_type)
    allPLInteractions = pd.concat([allPLInteractions, _df], ignore_index=True)

In [81]:
print(allPLInteractions.head(5))

  RESNR RESTYPE RESCHAIN RESNR_LIG RESTYPE_LIG RESCHAIN_LIG  DIST  \
0     7     PHE        A       591         LIG            Z  3.85   
1     9     VAL        A       591         LIG            Z  3.54   
2   103     ILE        A       591         LIG            Z  3.57   
3   107     TYR        A       591         LIG            Z  3.84   
4   107     TYR        A       591         LIG            Z  3.78   

   LIGCARBONIDX  PROTCARBONIDX                   LIGCOO  ... METAL_TYPE  \
0        6740.0          112.0   (6.689, 1.326, 30.431)  ...        NaN   
1        6741.0          149.0   (8.517, 1.116, 28.027)  ...        NaN   
2        6734.0         1627.0  (10.154, 3.047, 23.083)  ...        NaN   
3        6734.0         1696.0  (10.154, 3.047, 23.083)  ...        NaN   
4        6727.0         1694.0   (6.907, 3.074, 26.738)  ...        NaN   

  TARGET_IDX TARGET_TYPE COORDINATION LOCATION  RMS  GEOMETRY COMPLEXNUM  \
0        NaN         NaN          NaN      NaN  NaN       

In [82]:
allPLInteractions.to_csv("allPLI.csv")

In [83]:
protlig = PDBComplex()
protlig.load_pdb("11gs.pdb")  # load the pdb file
type(protlig)

plip.structure.preparation.PDBComplex

In [84]:
list(protlig.atoms.values())[0].coords

(30.676, 3.06, 38.855)

In [85]:
import openbabel as ob

def find_atom_by_coordinates(molecule, target_coordinates):
    for atom in list(molecule.atoms.values()):
        atom_coordinates = atom.coords

        # Compare the coordinates
        if round(atom_coordinates[0], 3) == round(target_coordinates[0], 3) and \
                round(atom_coordinates[1], 3) == round(target_coordinates[1], 3) and \
                round(atom_coordinates[2], 3) == round(target_coordinates[2], 3):
            return atom

    # If no match found
    return None

In [86]:
# Find the atom with the target coordinates
target_coords = (8.517, 1.116, 28.027)
target_atom = find_atom_by_coordinates(protlig, target_coordinates=list(target_coords))

# Print the atom information
if target_atom is not None:
    print('Atom Found:')
    print('Atom Type:', target_atom.type)
    print('Atom Index:', target_atom.idx)
    print('Atom Coordinates:', target_atom.coords)
else:
    print('Atom Not Found.')

Atom Found:
Atom Type: C3
Atom Index: 6739
Atom Coordinates: (8.517, 1.116, 28.027)


##  Import node and edge features

In [ ]:
import pickle

with open("../features/PLComplex.pkl", 'rb') as file:
    totalDict = pickle.load(file)

repGraph = totalDict['repDict']['9icd_1']
